In [ ]:
import pandas as pd
import os
import datetime

In [ ]:
fires = pd.read_csv("./data/Fires_data/incendios2015.csv", sep=';' , encoding="utf-8",na_values="nan")
fires.head()

In [ ]:
fires.columns

In [ ]:
incendio = fires['incendio']==1
falso_alarme = fires['falso_alarme']==0 
falso_alarme_tipo = fires['tipo']!='Falso Alarme'
fogacho = fires['fogacho']==0
fires = fires[ incendio & falso_alarme & falso_alarme_tipo & fogacho]
fires.drop([
               'ano', 
               'codigo_sgif',
               'codigo_anpc',
               'data_primeira_intervencao',
               'hora_primeira_intervencao',
               'fonte_alerta',
               'nut',
               'ine',
               'causa', 'reacendimento' , 'agricola',
               'queimada','falso_alarme','fogacho','incendio','perimetro','aps', 'regiao_prof', 'ugf' 
          ],axis=1, inplace=True)
fires = fires.sort_values('area_total',  ascending=False)
fires.dropna()
fires.head()

In [ ]:
def find_images(coords,sensing_date_FROM,sensing_date_TO,filename) :
    global USERNAME
    global PASSWORD
    command = "dhusget.sh -u "+USERNAME+" -p "+PASSWORD +" -S "+sensing_date_FROM+" -E "+sensing_date_TO +" -c "+coords+ " -m Sentinel-2"
    command = command + " -q " + filename +  ".xml -C " + filename + ".csv"
    os.system(command)

In [ ]:
def date_to_iso_format (date,before=True):
    date = str(date)
    if (date=="nan") :
        return "nan"
    splited = date.split('/')
    dt = datetime.datetime(int(splited[2]), int(splited[1]),  int(splited[0]))
    if before : 
       dt = dt - datetime.timedelta(days=1)
    else :
       dt = dt + datetime.timedelta(days=1)
    
    iso = str(dt.year) + "-" + str(dt.month) + "-" + str(dt.day) + "T06:00:00.000Z"
    return iso
date_to_iso_format("07/08/2015")

In [ ]:
def dms2dd(degrees, minutes, seconds, direction):
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction == 'E' or direction == 'N':
        dd *= -1
    return dd;
print(dms2dd(8,17,1.2876,'N'))
dms2dd(39,56,36.3696,'W')

In [ ]:
def dms_to_decimal(text,direction):
    splited = text.split(':')
    degrees = splited[0]
    minutes = splited[1]
    if len(splited) < 3 : 
        return "0"
    else :
        seconds = splited[2].replace('\'', "")
    return dms2dd(degrees, minutes, seconds, direction)
print(dms_to_decimal("40:33:26.5068000000082''" , 'W'))
dms_to_decimal( "7:41:29.922000000001'' ", 'N' )

In [ ]:
USERNAME = ''
PASSWORD = ''
lon1="-4"
lat1="30"
lon2="26"
lat2="46"
coords = lon1 + "," + lat1 + ":" + lon2 + "," + lat2 
sensing_date_FROM = "2015-01-01T06:00:00.000Z"
sensing_date_TO = "2016-01-01T06:00:00.000Z"
filename = "./images_with_fires_results/test"
#find_images(coords,sensing_date_FROM,sensing_date_TO,filename)

In [ ]:
head = fires.head(50)
for index, row in head.iterrows():
    lat =dms_to_decimal(row["lat"],"W")
    lon =dms_to_decimal(row["lon"],"N")
    lon1=str( float(lon)-0.5 )
    lat1=str( float(lat)-0.5 )
    lon2=str( float(lon)+0.5 )
    lat2= str( float(lat)+0.5 )
    coords = lon1 + "," + lat1 + ":" + lon2 + "," + lat2 
    sensing_date_FROM = date_to_iso_format(row["data_alerta"])
    sensing_date_TO = date_to_iso_format(row["data_extincao"],False)
    filename = "./images_with_fires_results/index"+str(index)
    print(coords,sensing_date_FROM,sensing_date_TO,filename)
    print("\n")
    find_images(coords,sensing_date_FROM,sensing_date_TO,filename)
    

In [ ]:
import glob
files = glob.glob("./images_with_fires_results/*.csv")

for file in files : 
    global USERNAME, PASSWORD
    if (os.path.getsize(file) >0 ) :
        downloads = pd.read_csv(file, sep=',' , encoding="utf-8",na_values="nan",header=None)
        splited = file.split('\\')
        index = splited[len(splited)-1][:-4]
        outputpath = "./data/Satellite_images/" + index
        for index, row in downloads.iterrows():
            outputpath = outputpath + '/' + row[0]
            command = "wget --content-disposition --continue --user="+USERNAME+" --password="+PASSWORD+" \""+ row[1]+"\" -o " + outputpath
            print(row[0],row[1],"\n\n")
            os.system(command)